# Image analyzer pipeline

In [1]:
%load_ext autoreload
%autoreload 2

import os
import numpy as np
import pytorch_lightning as pl
import torch
import matplotlib.pyplot as plt
from tqdm import tqdm
from skimage.io import imread
from skimage import measure
import pandas as pd
from scipy import stats
import random
from Unet_MCT import Unet

c:\Users\mauri\Desktop\Facultad\Tesis\Pasantia2\mastcell-data-main\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# support methods
from skimage import morphology
import numpy as np
import cv2

def getDividableNumber(n, divisor=64): # Esta funcion da la division entera, si hay resto da uno mas
    remainer = n % divisor
    if remainer == 0:
        return n

    return int(n + (divisor - remainer))

def zeroPad(img, size): # Le crea un borde a la imagen

    old_size = img.shape[:2]
    delta_w = abs(old_size[1] - size)
    delta_h = abs(old_size[0] - size)
    top, bottom = delta_h//2, delta_h-(delta_h//2)
    left, right = delta_w//2, delta_w-(delta_w//2)
    
    color = [0, 0, 0]
    new_img = cv2.copyMakeBorder(img, top, bottom, left, right, cv2.BORDER_REFLECT,value=color) # Crea un borde
    
    return new_img,[top, bottom, left, right]


def filterBySize(mask,mycMinDia,factor): # Da una mascara binaria con todos los objetos mayores a cierta area

    binaryImg=mask.copy()
    binaryImg[binaryImg>0]=1
    pxMinSize =(np.power((mycMinDia/factor/2),2)*np.pi) #~7ym² -> pixel area of a circle with the set diameter
    binaryImg = morphology.remove_small_objects(binaryImg.astype('bool'), pxMinSize, connectivity=8)
    copyMask=mask.copy()
    copyMask[binaryImg==0]=0
    return copyMask

In [3]:
#resShape has to be set depending on the image size -> default could lead to an unnecessary high amount of memory usage
def applyModeltoGetIdMask(model,img,resShape=-1,mycMinDia=3,factor=0.248,do_filter=False):
    if (resShape==-1):
        resShape=getDividableNumber(int(np.max(np.shape(img))))
    #prepare image
    imgResh,[top, bottom, left, right]=zeroPad(img.copy(),resShape)
    
    
    #apply model
    t_img=torch.tensor(imgResh.transpose(2,0,1)).unsqueeze(dim=0)
    t_img=t_img.to(device)
    seg=model(t_img).detach().requires_grad_(False).to('cpu').squeeze().numpy()
    seg=seg[top:resShape-bottom, left:resShape-right]
    seg[seg>=0.5]=1
    seg[seg<0.5]=0
    
    if(do_filter):
        seg=filterBySize(seg,mycMinDia,factor)
    
    labeled_seg, count_result_seg = measure.label(seg,connectivity=2, return_num=True)
    
    return labeled_seg

def getTopN(narray,perc=0.1):
    return np.sort(narray)[-int(len(narray)*perc):]

def equArea(diameter):
    return np.power((diameter/2.0),2)*np.pi

#minDia, factor and areaFactor has to be set depending on the imaging settings and image data
def measureImg(segResult,minDia=3,factor=0.248, areaFactor=0.061828822,trainNineQuantile=0,trainMedian=0,sol_highQuantiles=[],sol_lowQuantiles=[]):
    labeled_result, count_result = measure.label(segResult,connectivity=2, return_num=True)
    filtered_img=filterBySize(labeled_result,mycMinDia=minDia,factor=factor)
    filteredIds=list(np.unique(filtered_img))
    m=measure.regionprops(filtered_img)
    m=[[x.eccentricity,x.solidity,x.area*areaFactor,equArea(x.equivalent_diameter*factor)] for x in m]
    
    areas=[x[2] for x in m]
    
    #top n areas
    top_ten_area=getTopN(areas)
    
    ninety_quantile=np.quantile(areas, 0.90)
    
    #percentage above norm
    normQuantileTest=areas>trainNineQuantile
    normPercQuantile=(sum(normQuantileTest)/len(normQuantileTest))*100
    
    normMedianTest=areas>trainMedian
    normPercMedian=(sum(normMedianTest)/len(normMedianTest))*100
    
    #solidity
    sols=[x[1] for x in m]
    sol_measures=[]
    for sol_quant in sol_highQuantiles:
        solQuantileTest=sols>sol_quant
        sol_measures.append((sum(solQuantileTest)/len(solQuantileTest))*100)
    
    for sol_quant in sol_highQuantiles:
        solQuantileTest=sols<sol_quant
        sol_measures.append((sum(solQuantileTest)/len(solQuantileTest))*100)
    sol_measures.extend(sol_highQuantiles)
    
    for sol_quant in sol_lowQuantiles:
        solQuantileTest=sols<sol_quant
        sol_measures.append((sum(solQuantileTest)/len(solQuantileTest))*100)
    
    sol_measures.extend(sol_lowQuantiles)
    
    return np.concatenate([np.concatenate([np.concatenate([np.std(m,axis=0),np.mean(m,axis=0),np.median(m,axis=0),stats.skew(m,axis=0)]), [np.median(top_ten_area),np.mean(top_ten_area),ninety_quantile/np.median(areas),ninety_quantile/np.median([np.log(x) for x in areas]),normPercQuantile,normPercMedian,ninety_quantile,np.median([np.log(x) for x in areas]),np.median(areas),trainNineQuantile,trainMedian]]),np.array(sol_measures)]),areas

In [4]:
def select_device(verbose = True):
    is_cuda = torch.cuda.is_available()

    if is_cuda:
        device = torch.device("cuda")
        if verbose:
            print("GPU disponible")
    else:
        device = torch.device("cpu")
        if verbose:
            print("GPU no disponible, usando CPU")
    
    return device

# Initialize model

In [5]:
#coregir dict

old_state_dict = torch.load('inference_model.pth')
new_state_dict = {}

for key, value in old_state_dict.items():
    new_key = key.replace("model.encoder.", "model.encoder.model.")
    new_state_dict[new_key] = value

print(new_state_dict.keys())
torch.save(new_state_dict, "new_inference_model.pth")

dict_keys(['model.encoder.model.stem.conv.weight', 'model.encoder.model.stem.bn.weight', 'model.encoder.model.stem.bn.bias', 'model.encoder.model.stem.bn.running_mean', 'model.encoder.model.stem.bn.running_var', 'model.encoder.model.stem.bn.num_batches_tracked', 'model.encoder.model.s1.b1.conv1.conv.weight', 'model.encoder.model.s1.b1.conv1.bn.weight', 'model.encoder.model.s1.b1.conv1.bn.bias', 'model.encoder.model.s1.b1.conv1.bn.running_mean', 'model.encoder.model.s1.b1.conv1.bn.running_var', 'model.encoder.model.s1.b1.conv1.bn.num_batches_tracked', 'model.encoder.model.s1.b1.conv2.conv.weight', 'model.encoder.model.s1.b1.conv2.bn.weight', 'model.encoder.model.s1.b1.conv2.bn.bias', 'model.encoder.model.s1.b1.conv2.bn.running_mean', 'model.encoder.model.s1.b1.conv2.bn.running_var', 'model.encoder.model.s1.b1.conv2.bn.num_batches_tracked', 'model.encoder.model.s1.b1.se.fc1.weight', 'model.encoder.model.s1.b1.se.fc1.bias', 'model.encoder.model.s1.b1.se.fc2.weight', 'model.encoder.model.s

In [6]:
%%capture out
# model
device = select_device()
model=Unet()
model.load_state_dict(torch.load('new_inference_model.pth'))
model.to(device)
model.eval()


# Create Directory for Output

In [7]:
outputPath='./output/'
if not os.path.exists(outputPath):
    os.makedirs(outputPath)


# Set up data path

In [8]:
path_to_data ="./Modelling_Dataset/png/"

files=os.listdir(path_to_data)
files=[x for x in files if 'label' not in x]

# Load measures

In [9]:
# these measures not only include the used measures mentioned in the publication, but also additional measures used for further testing
# the following measures contain the 90 quantile as well as the median of the nuclei areas in the whole training data as well as the solidity measures of different quantiles
trainNineQuantile = np.load('./measures/90_quantile_allTrain.npy')
trainMedian = np.load('./measures/median_allTrain.npy')*2

sol_ninety = np.load('./measures/sol_ninety.npy')
sol_ninetyfive = np.load('./measures/sol_ninetyfive.npy')
sol_ninetyeight = np.load('./measures/sol_ninetyeight.npy')
sol_ten = np.load('./measures/sol_ten.npy')
sol_five = np.load('./measures/sol_five.npy')
sol_two = np.load('./measures/sol_two.npy')

sol_highQuantiles=[sol_ninety,sol_ninetyfive,sol_ninetyeight]
sol_lowQuantiles=[sol_ten,sol_five,sol_two]

# Create filelist

In [10]:
multipleFilesPerCase=False # required if there a more than one slide per file -> separator has to be set correctly depending on the file naming
sep='_'
if(multipleFilesPerCase):
    filesById=[x.split(sep)[0] for x in files]
else:
    filesById=files

filesById=np.unique(filesById)
fileMap={}
for file in filesById:
    fileMap[file]=[x for x in files if file in x]

# Apply Model and create measures

In [11]:
def divide_and_conquer(img, divs):
    h, w, _ = img.shape
    divh = h//divs[0]
    divw = w//divs[0]
    output = np.zeros((h, w))
    for i in range(divs[0]):
        for j in range(divs[1]):
            imgk = img[divh*i:divh*(i+1), divw*j:divw*(j+1)]
            outputk = applyModeltoGetIdMask(model,imgk,resShape=-1,do_filter=True)
            output[divh*i:divh*(i+1), divw*j:divw*(j+1)] = outputk
    
    output[output>0] = 1
    return output

In [12]:
combinedFileMeasures={}
fileMeasures={}
areas={}
nucleiCount={}
for file in tqdm(filesById):
    areas[file]=[]
    cropMeasures=[]
    cropNuclei=[]
    for crop in fileMap[file]:
        #load file -> has to be changed for different datatypes
        imgRaw = imread(path_to_data+crop)[:1200,:1600,:3]
        img=imgRaw.astype('float32')/255.0
        
        output = divide_and_conquer(img, (2, 2))

        #measure image
        cropM,a=measureImg(output,minDia=3,factor=0.248,trainNineQuantile=trainNineQuantile,trainMedian=trainMedian,sol_highQuantiles=sol_highQuantiles,sol_lowQuantiles=sol_lowQuantiles)
        areas[file].extend(a)
        cropMeasures.append(cropM)
        fileMeasures[crop]=cropM
        
        cropNuclei.append(len(np.unique(output))-1)
        
        #recolored output -> just for visualization
        #for idd in list(np.unique(output))[1:]:
        #    output[np.where(output==idd)]=random.randint(127,255)
        
        fig, ax = plt.subplots(dpi=300)
        ax.set_xticks([], [])
        ax.set_yticks([], [])
        ax.imshow(np.array(img.squeeze()))
        ax.imshow(output, cmap='jet', alpha=0.4)
        plt.savefig(outputPath+file+'.png', bbox_inches='tight', pad_inches=0)
        plt.close()
        
        
    nucleiCount[file]=np.mean(cropNuclei)
    combinedFileMeasures[file]=np.mean(cropMeasures,axis=0)

  6%|▌         | 5/85 [00:48<12:57,  9.71s/it]


KeyboardInterrupt: 

# Setup columns for measured values

In [ ]:
m=['eccentricity','solidity','mu_area','mu_equArea']
c=['std','mean', 'median', 'skew']

In [ ]:
cols=[]
cols.append('file')
for cc in c:
    for mm in m:
        cols.append('_'.join([cc,mm]))

In [ ]:
cols.extend(['median_top_10%','mean_top_10%','90%quantile/median(areas)','90%quantile/median(log(areas))','% above 90 Quantile (Training Data Split)','% above double Median (Training Data Split)','90 Quantile','median(log(areas))','median(areas)','trainNinetyQuantile','doubleMedian','% above_solidity_ninetyQuant','% above_solidity_ninetyfiveQuant','% above_solidity_ninetyeightQuant','% below_solidity_ninetyQuant','% below_solidity_ninetyfiveQuant','% below_solidity_ninetyeightQuant','ninetyQuant','ninetyfiveQuant','ninetyeightQuant','% below_solidity_tenQuant','% below_solidity_fiveQuant','% below_solidity_twoQuant','tenQuant','fiveQuant','twoQuant'])

# Setup dataframes

In [ ]:
singleMeasureDF = pd.DataFrame(columns=cols)
i=0
for f in fileMeasures.keys():
    
    entry=[f]
    entry.extend(fileMeasures[f])
    
    singleMeasureDF.loc[i] = entry
    i=i+1

In [ ]:
if(multipleFilesPerCase):
    combinedMeasureDF = pd.DataFrame(columns=cols)
    i=0
    for f in combinedFileMeasures.keys():

        entry=[f]
        entry.extend(combinedFileMeasures[f])

        combinedMeasureDF.loc[i] = entry
        i=i+1

# Save dataframe als Excel File

In [ ]:
# Create excel output
writer = pd.ExcelWriter(outputPath+'output.xlsx', engine='xlsxwriter')

if(multipleFilesPerCase):
    combinedMeasureDF.to_excel(writer, sheet_name='Case', index=False)
singleMeasureDF.to_excel(writer, sheet_name='Files', index=False)

writer.close()